In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('csv/mobile01.csv')

In [2]:
df

,URL,Date,Title,Author,Content
0,topicdetail.php?f=291&t=5775819,2019-04-23 15:02,[公告] 數位銀行/電子支付「分享碼」、「折扣碼」統一集中討論處。,\n Mobile01系統服務員\n ...,\n 各位 投資理財 版的會員...
1,topicdetail.php?f=291&t=4369995,2015-05-04 8:57,金融股-長期投資交流園地,\n weber2654\n,\n\n\n\n\n\n［2019/6/10修改］\n\n股東會陸續召開，除權息時期的考量表...
2,topicdetail.php?f=291&t=5994635,2020-01-04 12:32,蝦蝦果園集中串,\n 杜師傅\n,\n 蝦皮的一個活動邀請大家互...
3,topicdetail.php?f=291&t=5861631,2019-08-02 13:51,109年3月【如果認為會影響獲利請勿進入觀看】空頭分流,\n jiangbruce888\n ...,"\n 祝大家休假愉快,今天是空..."
4,topicdetail.php?f=291&t=6151367,2020-07-27 9:19,台積電漲停..天有異象,\n 古錐阿土伯\n,\n 是福???????\n是...
...,...,...,...,...,...
29967,topicdetail.php?f=291&t=5060717,2017-02-07 21:21,有app有提供以下功能嗎?,\n blackpitduke\n ...,\n 1.開盤前與收盤前試搓跌...
29968,topicdetail.php?f=291&t=5054296,2017-01-31 9:36,請問大家.設定的日期未到.銀行可以扣款嗎?,\n choutoung\n,\n 大家好:不知這問題是在這...
29969,topicdetail.php?f=291&t=448373,2007-11-20 1:23,有人知道美安這個直銷公司嗎??,\n pjj09\n,\n 最近表哥加入了美安這個直...
29970,topicdetail.php?f=291&t=5059721,2017-02-06 21:31,買房最佳時機,\n 股市大新手\n,\n 如主題，各位認為現在是入...


In [3]:
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
ws = WS("./data")
pos = POS("./data")
ner = NER("./data")

In [31]:
arr = df['Content'][5:20]
sentence_list = arr.tolist()

In [11]:
word_sentence_list = ws(
    sentence_list,
    sentence_segmentation = True, # To consider delimiters
    segment_delimiter_set = {",", "。", ":", "?", "!", ";"}) # This is the defualt set of delimiters
    # recommend_dictionary = dictionary1, # words in this dictionary are encouraged
    # coerce_dictionary = dictionary2, # words in this dictionary are forced
#)

pos_sentence_list = pos(word_sentence_list)

entity_sentence_list = ner(word_sentence_list, pos_sentence_list)

In [ ]:
pos_sentence_list[0]

In [ ]:
def print_word_pos_sentence(word_sentence, pos_sentence):
    assert len(word_sentence) == len(pos_sentence)
    for word, pos in zip(word_sentence, pos_sentence):
        print(f"{word}({pos})", end="\u3000")
    print()
    return
    
for i, sentence in enumerate(sentence_list):
    print()
    print(f"'{sentence}'")
    print_word_pos_sentence(word_sentence_list[i],  pos_sentence_list[i])
    for entity in sorted(entity_sentence_list[i]):
        print(entity)

In [32]:
import re

corpus = sentence_list

collect_corpus = []
for i in corpus:
    clean_c = re.sub('[【】《》「」]', '', i)
    clean_c = clean_c.replace(' ','')
    clean_c = clean_c.replace('\n','')
    clean_c = clean_c.replace('\u3000','')
    if len(clean_c) > 0:
        collect_corpus.append(clean_c)


In [33]:
word_segment = ws(collect_corpus,
                  sentence_segmentation=True,
                  segment_delimiter_set={'?', '？', '!', '！', '。', '.', ',', '，', ';', ':', '、'})

In [ ]:
word_segment[0]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

cut_corpus = []
for i in word_segment:
    cut_corpus.append(' '.join(i))

text_cv = CountVectorizer()

td_matrix = text_cv.fit_transform(cut_corpus)

print (td_matrix.shape)

print (text_cv.vocabulary_.keys())

In [35]:
tfidf = TfidfTransformer()

tfidf_matrix = tfidf.fit_transform(td_matrix)

tfidf.idf_

array([3.07944154, 3.07944154, 3.07944154, ..., 3.07944154, 3.07944154,
       3.07944154])

In [36]:
import pandas as pd
df = pd.DataFrame(tfidf_matrix.T.toarray(), index=text_cv.vocabulary_.keys())

In [37]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
黃金,0.088653,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
概況,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.047616,0.0,0.000000,0.0,0.0,0.0,0.0
方面,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.022331,0.0,0.0,0.0,0.0
週一,0.017731,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
7月,0.035461,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0


In [40]:
df.shape

(2216, 15)

In [38]:
df2 = pd.DataFrame(tfidf_matrix.toarray(), columns=text_cv.vocabulary_.keys())

In [39]:
df2.head()

,黃金,概況,方面,週一,7月,27日,亞市,早盤,現貨,高位,...,趕緊,廠房,賣錢,設廠,手機會,用品,家電,電動,汽車,留在
0,0.088653,0.0,0.0,0.017731,0.035461,0.017731,0.0,0.017731,0.02491,0.0,...,0.0,0.336881,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
df_sum = pd.DataFrame(df.sum(axis=1))
# df_sum.sort_values(by=0)

In [55]:
df["sum"] = df.sum(axis=1)

In [58]:
df.sort_values(by = ['sum'], ascending = False)
# top 5 keywords: 人口 網站 終於 出去 看壞

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,sum
人口,0.000000,0.0,0.0,0.000000,0.0,0.0,0.059249,0.014512,0.234136,0.0,0.596077,0.0,0.000000,0.0,0.000000,0.903973
網站,0.000000,0.0,0.0,0.808922,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.808922
終於,0.052588,0.0,0.0,0.000000,0.0,0.0,0.016678,0.490176,0.020175,0.0,0.013246,0.0,0.000000,0.0,0.138708,0.731570
出去,0.125456,0.0,0.0,0.000000,0.0,0.0,0.000000,0.332213,0.074384,0.0,0.028728,0.0,0.000000,0.0,0.150413,0.711195
看壞,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.669583,0.0,0.000000,0.669583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
直到,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.006802,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.006802
變成,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.006802,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.006802
老者,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.006802,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.006802
紐約,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.006802,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.006802
